In [18]:
from keras.models import load_model
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.metrics import categorical_accuracy
import pickle
from libs.easeml import evalAccuracy

In [19]:
#Ugly Example preprocessing
pad_char=0
start_char=1
oov_char=2
index_from=3

with open('models/wordDictionary.pkl', 'rb') as handle:
    wordDict = pickle.load(handle)

def encodeList(text_list,vocab_size):
    encoded=[start_char]
    for w in text_list:
        val=wordDict.get(w,oov_char-index_from)+index_from
        if val>=vocab_size:
            val=vocab_size-1
        encoded.append(val)
    #print(encoded)
    return encoded
       
def encodeData(data,vocab_size = 5000):
    data_out=[]
    for index, row in data.iterrows():
        text=(row.iloc[0]+' ### '+row.iloc[1]+' ### '+row.iloc[2])
        text_list=text.split()
        encoded=encodeList(text_list,vocab_size)
        data_out.append(encoded)
    return np.array(data_out)
def encodedLabels(data):
    data_out=[]
    for index, row in data.iterrows():
        if row.iloc[0] == 'others':
            data_out.append([1,0,0,0])
        elif row.iloc[0] == 'happy':
            data_out.append([0,1,0,0])
        elif row.iloc[0] == 'angry':
            data_out.append([0,0,1,0])
        elif row.iloc[0] == 'sad':
            data_out.append([0,0,0,1])
        else:
            raise Exception()
    return np.array(data_out).astype(int)
def prepare_data(x_data,max_conv_length):
    x_data = sequence.pad_sequences(x_data, maxlen=max_conv_length)
    return x_data
def loadAndPrepareData(fname,vocab_size=5000,max_conv_length=50):
    data = pd.read_csv(fname,sep='	',header=None)
    x_data_raw=data.filter([1,2,3], axis=1).copy(deep=True)
    y_data_raw=data.filter([4], axis=1).copy(deep=True)  
    x_data = encodeData(x_data_raw,vocab_size)
    y_data = encodedLabels(y_data_raw)
    x_data = prepare_data(x_data,max_conv_length)                       
    return x_data,y_data

In [20]:
fname="data_dev/dev.txt"
x_data,y_data=loadAndPrepareData(fname)

[1, 274, 102, 146, 11, 3, 3591, 276, 4999, 605, 3300, 1571, 176, 4381, 4999, 3, 1188, 605, 276, 4999, 2600, 4521]
[1, 2, 283, 403, 113, 6, 6, 3, 20, 283, 4, 16, 7, 3, 487, 4, 558, 4999, 6, 6]
[1, 907, 5, 33, 71, 108, 3, 38, 5, 33, 108, 41, 4, 502, 223, 3, 5, 58, 46, 675]
[1, 4999, 2054, 11, 117, 3, 483, 21, 126, 1513, 3, 2]
[1, 21, 19, 4999, 3, 5, 133, 438, 4999, 7, 3, 303, 19, 4999, 11]
[1, 4999, 17, 146, 27, 711, 6, 6, 3, 504, 82, 752, 5, 711, 131, 10, 3, 228, 17, 70, 11, 62, 53, 81, 74, 4219, 6, 6]
[1, 5, 133, 89, 3, 44, 101, 5, 321, 4, 5, 33, 23, 6, 3, 394, 8, 317]
[1, 221, 5, 916, 608, 3, 55, 405, 6, 3, 85, 5, 47, 63]
[1, 1861, 81, 222, 3, 4, 4999, 4, 64, 954, 438, 32, 1363, 4, 166, 97, 80, 476, 4, 766, 6, 3, 6]
[1, 413, 30, 553, 20, 1300, 3, 5, 49, 67, 21, 4999, 7, 3, 35, 64, 473]
[1, 241, 329, 4999, 3, 184, 4, 2856, 3, 4999]
[1, 23, 463, 3, 347, 41, 20, 130, 6, 3, 53, 27, 222, 8, 1403, 186]
[1, 5, 47, 8, 289, 20, 363, 3, 101, 4, 49, 1528, 20, 363, 32, 96, 4, 40, 18, 1528, 20, 36

[1, 460, 4, 3, 460, 11, 6, 3, 2429]
[1, 1004, 5, 327, 138, 3532, 12, 4, 6, 3, 12, 360, 23, 7, 3, 2]
[1, 4999, 3, 62, 105, 19, 21, 7, 3, 5, 306, 97, 3900]
[1, 5, 58, 539, 3, 40, 17, 144, 679, 21, 376, 3, 4999, 26, 6]
[1, 44, 24, 34, 6, 10, 3, 4, 34, 116, 10, 10, 3, 621]
[1, 85, 21, 30, 991, 445, 1657, 13, 9, 449, 3, 35, 40, 8, 263, 21, 167, 358, 7, 25, 165, 3, 195, 725]
[1, 78, 61, 747, 11, 1655, 10, 3, 503, 42, 4, 16, 22, 1064, 4, 64, 83, 5, 262, 561, 3, 4999, 10]
[1, 5, 124, 18, 80, 39, 759, 3, 55, 19, 31, 6, 3, 4999, 17, 293]
[1, 55, 19, 21, 110, 3, 159, 56, 63, 3, 155, 572]
[1, 357, 5, 285, 20, 279, 12, 386, 12, 922, 3290, 20, 3662, 4574, 3, 4999, 159, 48, 3, 21, 51, 1419]
[1, 152, 129, 3, 199, 49, 46, 14, 129, 3, 78, 61, 27, 164, 129]
[1, 77, 45, 4, 3, 28, 4999, 2828, 714, 32, 4449, 2356, 4243, 217, 3, 145, 16, 4, 217, 82, 6]
[1, 86, 30, 23, 31, 106, 13, 9, 29, 12, 57, 9, 29, 12, 76, 3, 367, 26, 21, 4999, 39, 106, 22, 21, 441, 75, 41, 39, 634, 7, 3, 738, 305]
[1, 5, 88, 163, 17, 70

[1, 35, 2602, 3, 5, 2602, 5, 42, 4, 69, 7, 3, 35, 2043, 145, 8, 222]
[1, 393, 81, 15, 111, 8, 204, 3, 95, 5, 998, 1871, 43, 3, 420]
[1, 3777, 24, 299, 3, 116, 3, 354, 247, 965, 32]
[1, 1052, 3, 44, 364, 6, 3, 274, 887, 192, 6]
[1, 122, 3, 91, 4, 40, 212, 6, 3, 35, 15, 2]
[1, 23, 82, 83, 3, 4999, 15, 126, 25, 161, 3, 26, 19, 31, 6]
[1, 3327, 367, 4999, 32, 7, 16, 17, 34, 21, 6, 3, 351, 5, 52, 21, 10, 10, 10, 3978, 199, 3, 179, 373, 99, 5, 654, 21, 5, 51, 1535, 593, 389]
[1, 412, 10, 5, 258, 49, 133, 17, 4317, 4999, 689, 23, 4317, 286, 3, 831, 286, 4999, 204, 131, 25, 148, 43, 3, 13, 9, 29, 12, 1360, 13, 9, 50, 118]
[1, 2, 5, 42, 943, 3, 122, 5, 4999, 943, 1749, 7, 3, 210, 19, 36, 629, 4999]
[1, 5, 40, 65, 256, 3, 5, 40, 123, 245, 3, 175, 32, 26, 19, 36, 551, 658]
[1, 35, 52, 11, 3, 408, 19, 14, 216, 557, 7, 115, 5, 708, 42, 4, 7, 3, 334, 4, 2]
[1, 31, 30, 1396, 190, 259, 17, 8, 149, 11, 92, 2823, 1741, 39, 329, 32, 3, 630, 152, 3034, 22, 383, 241, 4999, 3034, 4999, 3, 356, 532, 669, 499

In [4]:
outpath='models/'

In [5]:
model=load_model(outpath+"model.h5")

W0720 23:17:34.596773 139902536447808 deprecation_wrapper.py:119] From /home/leo/ETH/DS3/gtWorkshop/sandbox/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0720 23:17:34.608963 139902536447808 deprecation_wrapper.py:119] From /home/leo/ETH/DS3/gtWorkshop/sandbox/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0720 23:17:34.620575 139902536447808 deprecation_wrapper.py:119] From /home/leo/ETH/DS3/gtWorkshop/sandbox/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0720 23:17:34.821377 139902536447808 deprecation_wrapper.py:119] From /home/leo/ETH/DS3/gtWorkshop/sandbox/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_sessi

In [6]:
scores = model.evaluate(x_data, y_data, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

2755/2755 [==============================] - 1s 202us/step
Accuracy: 83.81%


In [7]:
y_pred=model.predict(x_data)

In [8]:
evalAccuracy(y_data,y_pred,categorical_accuracy)

Your score 83.81124877929688


In [23]:
x_data[2]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   1, 907,   5,  33,  71, 108,   3,  38,   5,
        33, 108,  41,   4, 502, 223,   3,   5,  58,  46, 675], dtype=int32)

In [24]:
y_data[2]

array([0, 1, 0, 0])